![NYC Skyline](../images/airbnb/nyc.jpg)

Welcome to New York City, one of the most-visited cities in the world. There are many Airbnb listings in New York City to meet the high demand for temporary lodging for travelers, which can be anywhere between a few nights to many months. In this project, we will take a closer look at the New York Airbnb market by combining data from multiple file types like `.csv`, `.tsv`, and `.xlsx`.

Recall that **CSV**, **TSV**, and **Excel** files are three common formats for storing data. 
Three files containing data on 2019 Airbnb listings are available to you:

**data/airbnb_price.csv**
This is a CSV file containing data on Airbnb listing prices and locations.
- **`listing_id`**: unique identifier of listing
- **`price`**: nightly listing price in USD
- **`nbhood_full`**: name of borough and neighborhood where listing is located

**data/airbnb_room_type.xlsx**
This is an Excel file containing data on Airbnb listing descriptions and room types.
- **`listing_id`**: unique identifier of listing
- **`description`**: listing description
- **`room_type`**: Airbnb has three types of rooms: shared rooms, private rooms, and entire homes/apartments

**data/airbnb_last_review.tsv**
This is a TSV file containing data on Airbnb host names and review dates.
- **`listing_id`**: unique identifier of listing
- **`host_name`**: name of listing host
- **`last_review`**: date when the listing was last reviewed

## Importing Libraries

In [7]:
import numpy as np
import pandas as pd
from functools import reduce

## Loading Data

In [8]:
df_csv = pd.read_csv('../datasets/airbnb/airbnb_price.csv')
df_tcsv = pd.read_csv('../datasets/airbnb/airbnb_last_review.tsv',sep='\t')
df_xlsx = pd.read_excel('../datasets/airbnb/airbnb_room_type.xlsx')

In [9]:
# checking out the contents
display(df_csv.head(3))
display(df_tcsv.head(3))
display(df_xlsx.head(3))

,listing_id,price,nbhood_full
0,2595,225 dollars,"Manhattan, Midtown"
1,3831,89 dollars,"Brooklyn, Clinton Hill"
2,5099,200 dollars,"Manhattan, Murray Hill"


,listing_id,host_name,last_review
0,2595,Jennifer,May 21 2019
1,3831,LisaRoxanne,July 05 2019
2,5099,Chris,June 22 2019


,listing_id,description,room_type
0,2595,Skylit Midtown Castle,Entire home/apt
1,3831,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt


## Merging Data

In [10]:
# merging dataframes together for ease of use
df_list = [df_csv,df_tcsv,df_xlsx]

df = reduce(lambda  left,right: pd.merge(left,right,on=['listing_id']), df_list)
df.head()

,listing_id,price,nbhood_full,host_name,last_review,description,room_type
0,2595,225 dollars,"Manhattan, Midtown",Jennifer,May 21 2019,Skylit Midtown Castle,Entire home/apt
1,3831,89 dollars,"Brooklyn, Clinton Hill",LisaRoxanne,July 05 2019,Cozy Entire Floor of Brownstone,Entire home/apt
2,5099,200 dollars,"Manhattan, Murray Hill",Chris,June 22 2019,Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
3,5178,79 dollars,"Manhattan, Hell's Kitchen",Shunichi,June 24 2019,Large Furnished Room Near B'way,private room
4,5238,150 dollars,"Manhattan, Chinatown",Ben,June 09 2019,Cute & Cozy Lower East Side 1 bdrm,Entire home/apt


## Data Validation

Now, we are going to validate the types of each column.

In [11]:
# printing out the dtypes of each column so we can see which ones we will need to change
df.dtypes

listing_id      int64
price          object
nbhood_full    object
host_name      object
last_review    object
description    object
room_type      object
dtype: object

Upon inspecting each column, we see that...
- price should probable be converted to int type.
- room_type should most likely be a category to best represent the group
- last_review should be converted to datatime to make manipulating the row and viz. easier

In [12]:
# converting price column
df['price'] = df['price'].str.replace(' dollars','').astype('int')

Since we have multiple categories that are very similar, this was most likely an error when the data was being put in. When we look further, we see that the difference is only in capitalization. This means that if we convert all values in this column to lowercase, our consistency issues will be solved.

In [13]:
# converting room_type
df['room_type'].astype('category').value_counts()

room_type
Entire home/apt    8458
Private room       7241
entire home/apt    2665
private room       2248
ENTIRE HOME/APT    2143
PRIVATE ROOM       1867
Shared room         380
shared room         110
SHARED ROOM          97
Name: count, dtype: int64

In [14]:
# converting all values to lowercase
df['room_type'] = df['room_type'].str.lower()#.astype('category')

In [15]:
df['room_type'].value_counts()

room_type
entire home/apt    13266
private room       11356
shared room          587
Name: count, dtype: int64

In [16]:
# converting last_review to datetime
df['last_review'] = pd.to_datetime(df['last_review'])

In [17]:
# checking the datatypes of the columns now
df.dtypes

listing_id              int64
price                   int32
nbhood_full            object
host_name              object
last_review    datetime64[ns]
description            object
room_type              object
dtype: object

## Problems

### What are the dates of the earliest and most recent reviews? Store these values as two separate variables with your preferred names.

We have around a 7 month range in reviews

In [18]:
earliest = df['last_review'].min()
display(f"The earliest review in the DataFrame is: {earliest}")

most_recent = df['last_review'].max()
display(f"The most recent review in the DataFrame is: {most_recent}")

'The earliest review in the DataFrame is: 2019-01-01 00:00:00'

'The most recent review in the DataFrame is: 2019-07-09 00:00:00'

### How many of the listings are private rooms? Save this into any variable.

In [19]:
# selecting the rows where the room_type is equal to 'private room' and getting a count of those variables
num_priv_rooms = df[df['room_type'] == "private room"].shape[0]

### What is the average listing price? Round to the nearest penny and save into a variable.

In [20]:
average_price = df['price'].mean().round(2)
display(f"The average price of a list is ${average_price}")

'The average price of a list is $141.78'

### Combine the new variables into one DataFrame called review_dates with four columns in the following order: first_reviewed, last_reviewed, nb_private_rooms, and avg_price. The DataFrame should only contain one row of values.

In [21]:
# creating a data dict that we will later pass into a new DataFrame
data = {
    "first_reviewed": [earliest],
    "last_reviewed": [most_recent],
    "nb_private_rooms": [num_priv_rooms],
    "avg_price": [average_price]
}

review_dates = pd.DataFrame(data=data)
review_dates.head()

,first_reviewed,last_reviewed,nb_private_rooms,avg_price
0,2019-01-01,2019-07-09,11356,141.78
